In [5]:
from agents import Agent, Runner, function_tool, SQLiteSession
from dotenv import load_dotenv
from agents.extensions.visualization import draw_graph
import os
from pydantic import BaseModel
from agents.tracing import trace

load_dotenv()
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"

session = SQLiteSession("user_1", "dummy-agent-ai-memory/ai-memory.db")

class Answer(BaseModel):
    answer: str
    background_explanation: str

@function_tool
def get_weather(city: str) -> str:
    # Dummy implementation for weather fetching
    return "30"


geography_agent = Agent(
    name="Geography Agent",
    instructions="You are an expert in geography information.",
    handoff_description="Handles questions related to geography",
    tools=[get_weather],
    output_type=Answer,
)

economics_agent = Agent(
    name="Economics Agent",
    instructions="You are an expert in economics and financial data.",
    handoff_description="Handles questions related to economics",
)

main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing assistant. Transfer to the agent most capable of answering the user's requests.",
    handoffs=[
        geography_agent,
        economics_agent,
    ]
)

draw_graph(main_agent)

with trace("user_1"):
    result = await Runner.run(main_agent, "What is the second biggest city in Mongolia?", session=session,)

    print(result.last_agent.name)
    print(result.final_output)


Main Agent
The second biggest city in Mongolia is Erdenet.
